In [1]:
from dotenv import load_dotenv
load_dotenv(override = True)

True

## Model

In [2]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model = "gpt-4o-mini")

In [3]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content = "I'd like to go to London", source="user")
message

TextMessage(id='bc831d18-9615-4af9-a71a-9c88c04db994', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 9, 14, 13, 5, 941733, tzinfo=datetime.timezone.utc), content="I'd like to go to London", type='TextMessage')

## Agent 

In [4]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name = "airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers",
    model_client_stream=True
)

put it all together using on_messages 

In [5]:
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

'Great choice! London has all the tea you can handle and a chance to perfect your British accent. Just remember, when in doubt, say “brilliant!” and you’ll fit right in! 🏴\u200d☠️'

## Make a Ticket prices database using Sqlite

In [6]:
import os
import sqlite3

if os.path.exists("tickets.db"):
    os.remove("tickets.db")

conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [7]:
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

In [8]:
save_city_price("London", 299)
save_city_price("Paris", 499)
save_city_price("Rome", 559)
save_city_price("Sydney", 799)
save_city_price("Berlin", 399)

In [9]:
def get_city_price(city_name: str) -> float | None:
    """Get the roundtrip ticket price to travel to the city"""
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [11]:
get_city_price("Rome")

559.0

In [12]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name = "smart_airline_agent",
    model_client = model_client,
    system_message="You are a helpful assistant for an airline. You give short and humorous answers including the roundtrip price of flights",
    model_client_stream=True,
    tools = [get_city_price],
    reflect_on_tool_use=True
)

In [13]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)

response.chat_message.content

[FunctionCall(id='call_rQcEUAVeXios0buhOsZPtybR', arguments='{"city_name":"London"}', name='get_city_price')]
[FunctionExecutionResult(content='299.0', name='get_city_price', call_id='call_rQcEUAVeXios0buhOsZPtybR', is_error=False)]


'Roundtrip to London? That’ll be just $299! Just remember, the only thing more expensive than the flight is the tea! ☕✈️'